In [105]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from itertools import product
from sklearn.base import BaseEstimator, TransformerMixin
import pickle

In [106]:
input_vector = pd.read_csv('../data/data_input_vector.csv')
input_scalar = pd.read_csv('../data/data_input_scalar.csv')
input_general = pd.read_csv('../data/data_input_general_Info.csv')
output = pd.read_csv('../data/data_output.csv')

#### Data Preparation of data_input_general.csv

In [107]:
# DATA PREPARATION

# Drop unneeded columns
input_general = input_general.drop(columns=["VAN", "Index", "absolute_timestamp", 'stat_zeitpunkt_test'], axis=1)

# Replace 'BMW' with 'D'
input_general['fzg_verkaufsland_kfzint_kez'] = input_general['fzg_verkaufsland_kfzint_kez'].replace('BMW', 'D')

# Convert dates to timestamps
input_general['fzg_produktionsdatum'] = pd.to_datetime(input_general['fzg_produktionsdatum']).astype('int64')

# OMIT OUTLIER HANDLING

# PREPROCESSING
numerical_features = input_general.select_dtypes(include=['int64', 'float64']).columns
categorical_features = input_general.select_dtypes(include=['object']).columns

numerical_transformer = Pipeline(steps=[('scaler', MinMaxScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, numerical_features),
    ('cat', categorical_transformer, categorical_features)
])

X_general = preprocessor.fit_transform(input_general)
X_general = X_general.toarray()  # Convert sparse matrix to dense matrix

with open('../data/weekly_resampling/X_general.pkl', 'wb') as f:
    pickle.dump(X_general, f)

In [108]:
SPECIAL_VALUE = -1.0

#### Data Preparation of data_input_scalar.csv

In [109]:
# DATA PREPARATION

# Drop unneeded columns
input_scalar = input_scalar.drop(columns=["Unnamed: 0", "absolute_timestamp"], axis=1)

# Drop rows with NaN values
input_scalar = input_scalar.dropna()

# Set auslesedatum relative to fzg_produktionsdatum (TODO check compatibility with resampling)
#input_scalar['auslesedatum'] = pd.to_datetime(input_scalar['auslesedatum']).astype('int64')
#fzg_produktionsdatum_dict = input_general.set_index('VAN')['fzg_produktionsdatum'].to_dict()
#input_scalar['auslesedatum'] = input_scalar.apply(lambda row: row['auslesedatum'] - fzg_produktionsdatum_dict[row['VAN']], axis=1)

# OUTLIER HANDLING

class OutlierRemover(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=1.96):
        self.threshold = threshold
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_num = X.drop(columns=['auslesedatum'], axis=1).select_dtypes(include=['int64', 'float64'])
        z_scores = np.abs((X_num - X_num.mean()) / X_num.std())
        mask = (z_scores < self.threshold).all(axis=1)
        
        # Debug: Print the number of rows before and after outlier removal
        print("Rows before outlier removal:", X.shape[0])
        print("Rows after outlier removal:", X[mask].shape[0])
        
        return X[mask]
    
outlier_remover = OutlierRemover(threshold=1.96)
input_scalar = outlier_remover.fit_transform(input_scalar)

# RESAMPLING

# Convert dates to datetime
input_scalar['auslesedatum'] = pd.to_datetime(input_scalar['auslesedatum'])

# Sort values - this step is crucial for resampling
input_scalar = input_scalar.sort_values(by=['VAN', 'auslesedatum'])

# Group by 'VAN' and apply the resampling function
input_scalar = input_scalar.groupby('VAN').resample('W', on='auslesedatum').mean().reset_index()
input_scalar = input_scalar.dropna()

# Convert dates to timestamps
input_scalar['auslesedatum'] = pd.to_datetime(input_scalar['auslesedatum']).astype('int64')

# PREPROCESSING

numerical_features = input_scalar.select_dtypes(include=['int64', 'float64']).columns

# Sort by VAN and auslesedatum
input_scalar = input_scalar.sort_values(by=['VAN', 'auslesedatum'])

# Create 3D array with shape (num_vehicles, max_seq_len, dimension)
num_vehicles = output['VAN'].nunique()
max_seq_len_scalar = input_scalar.groupby('VAN').size().max()
dimension_scalar = len(numerical_features)
X_scalar = np.full((num_vehicles, max_seq_len_scalar, dimension_scalar), fill_value=SPECIAL_VALUE, dtype=np.float64)

# Scaling + Encoding
scaler = MinMaxScaler()
input_scalar[numerical_features] = scaler.fit_transform(input_scalar[numerical_features])

for van in input_scalar['VAN'].unique():
    van_data = input_scalar[input_scalar['VAN'] == van]
    input_sequence = van_data.drop(columns=['VAN']).values
    index = output.index[output['VAN'] == van].to_list()[0]
    seq_len = input_sequence.shape[0]
    X_scalar[index, 0:seq_len, :] = input_sequence
    
with open('../data/weekly_resampling/X_scalar.pkl', 'wb') as f:
    pickle.dump(X_scalar, f)

Rows before outlier removal: 167820
Rows after outlier removal: 154812


#### Data Preparation of data_input_vector.csv

In [110]:
# Drop unneeded columns
exclude_columns = ["Unnamed: 0", "absolute_timestamp", "BAUREIHE", 
                    "stat_zeit_temp_total_12_wert", "stat_zeit_temp_total_13_wert", 
                    "stat_zeit_temp_total_14_wert", "stat_hv_spannung_wert"]
input_vector = input_vector.drop(columns=exclude_columns, axis=1)

# Drop rows with NAN values
input_vector = input_vector.dropna()

# Convert dates to timestamps
input_vector['fzg_produktionsdatum'] = pd.to_datetime(input_vector['fzg_produktionsdatum']).astype('int64')

# Set auslesedatum relative to fzg_produktionsdatum (TODO check compatibility with resampling)
#input_vector['auslesedatum'] = pd.to_datetime(input_vector['auslesedatum']).astype(int)
#input_vector['auslesedatum'] = input_vector['auslesedatum'] - input_vector['fzg_produktionsdatum']

# OUTLIER HANDLING

class IQRBasedOutlierRemover(BaseEstimator, TransformerMixin):
    def __init__(self, factor=1.5):
        self.factor = factor
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_num = X.drop(columns=['auslesedatum'], axis=1).select_dtypes(include=['int64', 'float64'])
        Q1 = X_num.quantile(0.25)
        Q3 = X_num.quantile(0.75)
        IQR = Q3 - Q1
        mask = ~((X_num < (Q1 - self.factor * IQR)) | (X_num > (Q3 + self.factor * IQR))).any(axis=1)
        
        # Debug: Print the number of rows before and after outlier removal
        print("Rows before outlier removal:", X.shape[0])
        print("Rows after outlier removal:", X[mask].shape[0])
        
        return X[mask]

outlier_remover = IQRBasedOutlierRemover(factor=20)
input_vector = outlier_remover.fit_transform(input_vector)

# RESAMPLING

# Convert dates to datetime
input_vector['auslesedatum'] = pd.to_datetime(input_vector['auslesedatum'])

# Sort values - this step is crucial for resampling
input_vector = input_vector.sort_values(by=['VAN', 'auslesedatum'])

# Group by 'VAN' and apply the resampling function
input_vector = input_vector.groupby('VAN').resample('W', on='auslesedatum').mean().reset_index()
input_vector = input_vector.dropna()

# Convert dates to timestamps
input_vector['auslesedatum'] = pd.to_datetime(input_vector['auslesedatum']).astype('int64')

# PREPROCESSING

numerical_features = input_vector.select_dtypes(include=['int64', 'float64']).columns

# Sort by VAN and auslesedatum
input_vector = input_vector.sort_values(by=['VAN', 'auslesedatum'])

# Create 3D array with shape (num_vehicles, max_seq_len, dimension)
num_vehicles = output['VAN'].nunique()
max_seq_len_vector = input_vector.groupby('VAN').size().max()
dimension_vector = len(numerical_features)
X_vector = np.full((num_vehicles, max_seq_len_vector, dimension_vector), fill_value=SPECIAL_VALUE, dtype=np.float64)

# Scaling + Encoding
scaler = MinMaxScaler()
input_vector[numerical_features] = scaler.fit_transform(input_vector[numerical_features])

for van in input_vector['VAN'].unique():
    van_data = input_vector[input_vector['VAN'] == van]
    input_sequence = van_data.drop(columns=['VAN']).values
    index = output.index[output['VAN'] == van].to_list()[0]
    seq_len = input_sequence.shape[0]
    X_vector[index, 0:seq_len, :] = input_sequence
    
with open('../data/weekly_resampling/X_vector.pkl', 'wb') as f:
    pickle.dump(X_vector, f)

Rows before outlier removal: 95274
Rows after outlier removal: 74184
